In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("P_OPT_ACC_UPDATED.csv")

## REMOVAL OF SKUs WHOSE SRP = 0

In [3]:
data = df[df["SRP"] != 0]

In [4]:
data

,AS_OF_DATE,SKU,TTL_INV_DC,SRP,CHANGED_SRP,WEEKLY_SALES,TTL_INV_LOC,TTL_INT_LOC,DESCR,PARENT_MODEL,PHONE_MODEL,OEM
0,12/24/2022,4776R,0,79.99,79.99,0,0,0,APP GTB IOXCAN NONE BLK,,NONE,GTB
1,12/31/2022,4776R,0,79.99,79.99,0,0,0,APP GTB IOXCAN NONE BLK,UNASSIGNED,NONE,GTB
2,1/7/2023,4776R,0,79.99,79.99,0,0,0,APP GTB IOXCAN NONE BLK,UNASSIGNED,NONE,GTB
3,1/21/2023,4776R,0,79.99,79.99,0,0,0,APP GTB IOXCAN NONE BLK,UNASSIGNED,NONE,GTB
4,1/28/2023,4776R,0,79.99,79.99,0,0,0,APP GTB IOXCAN NONE BLK,UNASSIGNED,NONE,GTB
...,...,...,...,...,...,...,...,...,...,...,...,...
23040,3/11/2023,4344R,3069,29.99,29.99,244,8818,46,ATC SKT POPMAGSAF 805661 BLK,POP,805661,SKT
23041,3/18/2023,4344R,2714,29.99,29.99,292,8779,81,ATC SKT POPMAGSAF 805661 BLK,POP,805661,SKT
23042,3/25/2023,4344R,2409,29.99,29.99,261,8765,76,ATC SKT POPMAGSAF 805661 BLK,POP,805661,SKT
23043,4/1/2023,4344R,1898,29.99,29.99,289,8941,46,ATC SKT POPMAGSAF 805661 BLK,POP,805661,SKT


In [5]:
data['SKU'].nunique()

1159

In [6]:
d = data.groupby('SKU')['WEEKLY_SALES'].sum().sort_values(ascending=False).reset_index()

In [7]:
d

,SKU,WEEKLY_SALES
0,4924P,600518
1,4923P,272195
2,4630Q,251376
3,4559R,178115
4,4925P,107430
...,...,...
1154,4469R,0
1155,4811R,0
1156,4810R,0
1157,4809R,0


## REMOVAL OF SKUs WHOSE WEEKLY_SALES ARE = 0

In [8]:
d1 = d[d["WEEKLY_SALES"] != 0]

In [9]:
d1

,SKU,WEEKLY_SALES
0,4924P,600518
1,4923P,272195
2,4630Q,251376
3,4559R,178115
4,4925P,107430
...,...,...
772,4057Q,1
773,4602P,1
774,4003P,1
775,4600R,1


In [10]:
l = d1['SKU'].tolist()

In [11]:
l

['4924P',
 '4923P',
 '4630Q',
 '4559R',
 '4925P',
 '60075',
 '4229R',
 '4484R',
 '4436R',
 '4640Q',
 '4215R',
 '4224R',
 '4504Q',
 '4207R',
 '4297Q',
 '4590P',
 '4191R',
 '4282R',
 '4637Q',
 '4509Q',
 '4930P',
 '4194R',
 '4187R',
 '4682R',
 '4237R',
 '4144R',
 '4480R',
 '4621Q',
 '4631Q',
 '4937P',
 '4718R',
 '4936M',
 '4626Q',
 '4932P',
 '4190R',
 '4195R',
 '4627Q',
 '4180R',
 '4158R',
 '4625Q',
 '4285L',
 '4747R',
 '4513Q',
 '4203R',
 '4245R',
 '4638Q',
 '4934P',
 '4635Q',
 '4633Q',
 '4147R',
 '4926P',
 '4634Q',
 '4636Q',
 '4429R',
 '4248R',
 '4186R',
 '4226R',
 '4639Q',
 '4173R',
 '4430R',
 '4204R',
 '4686R',
 '4723R',
 '4244R',
 '4250R',
 '4219R',
 '4719R',
 '4746Q',
 '4225R',
 '4052Q',
 '4050R',
 '4188R',
 '4644Q',
 '4503Q',
 '4629Q',
 '4167R',
 '4710R',
 '4153R',
 '4247R',
 '4183R',
 '4931P',
 '4216R',
 '4283Q',
 '4532R',
 '4222R',
 '4936P',
 '4234R',
 '4155R',
 '4055Q',
 '4709R',
 '4800Q',
 '4717Q',
 '4677R',
 '4759P',
 '4176R',
 '4213R',
 '4148R',
 '4628Q',
 '4727R',
 '4051R',


In [12]:
l1 = df[df['SKU'].isin(l)]

In [13]:
l1

,AS_OF_DATE,SKU,TTL_INV_DC,SRP,CHANGED_SRP,WEEKLY_SALES,TTL_INV_LOC,TTL_INT_LOC,DESCR,PARENT_MODEL,PHONE_MODEL,OEM
17,10/8/2022,4898Q,1897,29.99,29.99,157,8822,99,ATC TLT MAGSAFSTD WALLET BLK,TYLT,WALLET,TLT
18,10/15/2022,4898Q,1699,29.99,29.99,187,8873,23,ATC TLT MAGSAFSTD WALLET BLK,TYLT,WALLET,TLT
19,10/22/2022,4898Q,1499,29.99,29.99,168,8849,42,ATC TLT MAGSAFSTD WALLET BLK,TYLT,WALLET,TLT
20,10/29/2022,4898Q,1576,29.99,29.99,141,8864,29,ATC TLT MAGSAFSTD WALLET BLK,TYLT,WALLET,TLT
21,11/5/2022,4898Q,1161,29.99,29.99,117,9075,30,ATC TLT MAGSAFSTD WALLET BLK,TYLT,WALLET,TLT
...,...,...,...,...,...,...,...,...,...,...,...,...
23040,3/11/2023,4344R,3069,29.99,29.99,244,8818,46,ATC SKT POPMAGSAF 805661 BLK,POP,805661,SKT
23041,3/18/2023,4344R,2714,29.99,29.99,292,8779,81,ATC SKT POPMAGSAF 805661 BLK,POP,805661,SKT
23042,3/25/2023,4344R,2409,29.99,29.99,261,8765,76,ATC SKT POPMAGSAF 805661 BLK,POP,805661,SKT
23043,4/1/2023,4344R,1898,29.99,29.99,289,8941,46,ATC SKT POPMAGSAF 805661 BLK,POP,805661,SKT


In [14]:
l1['SKU'].nunique()

777

In [15]:
m = l1.groupby('SKU')['AS_OF_DATE'].count().sort_values(ascending=True).reset_index()

In [16]:
m

,SKU,AS_OF_DATE
0,4818N,1
1,4269Q,1
2,4180Q,1
3,4185R,1
4,4182Q,1
...,...,...
772,4464R,27
773,4459R,27
774,4453Q,27
775,4493R,27


## REMOVAL OF SKUs WHOSE DATA IS LESS THAN 7 WEEKS INORDER FOR THE FORECASTING ALGORITHM

In [17]:
m1 = m[m['AS_OF_DATE'] > 7]

In [18]:
m1

,SKU,AS_OF_DATE
167,4883Q,8
168,4880Q,8
169,4862Q,8
170,4602P,8
171,4864Q,8
...,...,...
772,4464R,27
773,4459R,27
774,4453Q,27
775,4493R,27


In [19]:
n = m1['SKU'].tolist()

In [20]:
n1 = df[df['SKU'].isin(n)]

In [21]:
n1

,AS_OF_DATE,SKU,TTL_INV_DC,SRP,CHANGED_SRP,WEEKLY_SALES,TTL_INV_LOC,TTL_INT_LOC,DESCR,PARENT_MODEL,PHONE_MODEL,OEM
17,10/8/2022,4898Q,1897,29.99,29.99,157,8822,99,ATC TLT MAGSAFSTD WALLET BLK,TYLT,WALLET,TLT
18,10/15/2022,4898Q,1699,29.99,29.99,187,8873,23,ATC TLT MAGSAFSTD WALLET BLK,TYLT,WALLET,TLT
19,10/22/2022,4898Q,1499,29.99,29.99,168,8849,42,ATC TLT MAGSAFSTD WALLET BLK,TYLT,WALLET,TLT
20,10/29/2022,4898Q,1576,29.99,29.99,141,8864,29,ATC TLT MAGSAFSTD WALLET BLK,TYLT,WALLET,TLT
21,11/5/2022,4898Q,1161,29.99,29.99,117,9075,30,ATC TLT MAGSAFSTD WALLET BLK,TYLT,WALLET,TLT
...,...,...,...,...,...,...,...,...,...,...,...,...
23040,3/11/2023,4344R,3069,29.99,29.99,244,8818,46,ATC SKT POPMAGSAF 805661 BLK,POP,805661,SKT
23041,3/18/2023,4344R,2714,29.99,29.99,292,8779,81,ATC SKT POPMAGSAF 805661 BLK,POP,805661,SKT
23042,3/25/2023,4344R,2409,29.99,29.99,261,8765,76,ATC SKT POPMAGSAF 805661 BLK,POP,805661,SKT
23043,4/1/2023,4344R,1898,29.99,29.99,289,8941,46,ATC SKT POPMAGSAF 805661 BLK,POP,805661,SKT


## NOT CONSIDERING DC INVENTORY FOR FORECASTING

In [22]:
n1['TTL_INV_DC'].min()

-8791

In [23]:
n1['AS_OF_DATE'] = pd.to_datetime(n1['AS_OF_DATE'])

In [24]:
n1

,AS_OF_DATE,SKU,TTL_INV_DC,SRP,CHANGED_SRP,WEEKLY_SALES,TTL_INV_LOC,TTL_INT_LOC,DESCR,PARENT_MODEL,PHONE_MODEL,OEM
17,2022-10-08,4898Q,1897,29.99,29.99,157,8822,99,ATC TLT MAGSAFSTD WALLET BLK,TYLT,WALLET,TLT
18,2022-10-15,4898Q,1699,29.99,29.99,187,8873,23,ATC TLT MAGSAFSTD WALLET BLK,TYLT,WALLET,TLT
19,2022-10-22,4898Q,1499,29.99,29.99,168,8849,42,ATC TLT MAGSAFSTD WALLET BLK,TYLT,WALLET,TLT
20,2022-10-29,4898Q,1576,29.99,29.99,141,8864,29,ATC TLT MAGSAFSTD WALLET BLK,TYLT,WALLET,TLT
21,2022-11-05,4898Q,1161,29.99,29.99,117,9075,30,ATC TLT MAGSAFSTD WALLET BLK,TYLT,WALLET,TLT
...,...,...,...,...,...,...,...,...,...,...,...,...
23040,2023-03-11,4344R,3069,29.99,29.99,244,8818,46,ATC SKT POPMAGSAF 805661 BLK,POP,805661,SKT
23041,2023-03-18,4344R,2714,29.99,29.99,292,8779,81,ATC SKT POPMAGSAF 805661 BLK,POP,805661,SKT
23042,2023-03-25,4344R,2409,29.99,29.99,261,8765,76,ATC SKT POPMAGSAF 805661 BLK,POP,805661,SKT
23043,2023-04-01,4344R,1898,29.99,29.99,289,8941,46,ATC SKT POPMAGSAF 805661 BLK,POP,805661,SKT


## FORECASTS FOR WEEKLY_SALES ACROSS ALL LOCATIONS FOR ALL SKUs

In [25]:
forecasts = pd.DataFrame()

In [26]:
for sku in n1['SKU'].unique():
    # Filter the data for the specific SKU
    sku_data = n1[n1['SKU'] == sku]

    # Prepare the data for Prophet
    df_1 = sku_data.rename(columns={"AS_OF_DATE": "ds", "WEEKLY_SALES": "y"})
    df_1['ds'] = pd.to_datetime(df_1['ds'])
    df_1.sort_values('ds', inplace=True)

    # Create a Prophet model and fit it to the data
    model = Prophet()
    model.fit(df_1)

    # Generate future dates for forecasting (next 1 week)
    last_date = df_1['ds'].max()
    next_date = last_date + pd.DateOffset(weeks=1)
    nd = pd.DataFrame({'ds':[next_date]})

    # Perform the forecasting
    forecast = model.predict(nd)

    # Select only the forecasted values for the next 4 weeks
    forecast = forecast[['ds', 'yhat']]

    # Add the SKU-specific forecast to the overall forecasts DataFrame
    forecast['SKU'] = sku
    forecasts = pd.concat([forecasts, forecast])

00:23:02 - cmdstanpy - INFO - Chain [1] start processing
00:23:03 - cmdstanpy - INFO - Chain [1] done processing
00:23:03 - cmdstanpy - INFO - Chain [1] start processing
00:23:03 - cmdstanpy - INFO - Chain [1] done processing
00:23:03 - cmdstanpy - INFO - Chain [1] start processing
00:23:03 - cmdstanpy - INFO - Chain [1] done processing
00:23:03 - cmdstanpy - INFO - Chain [1] start processing
00:23:03 - cmdstanpy - INFO - Chain [1] done processing
00:23:03 - cmdstanpy - INFO - Chain [1] start processing
00:23:03 - cmdstanpy - INFO - Chain [1] done processing
00:23:03 - cmdstanpy - INFO - Chain [1] start processing
00:23:03 - cmdstanpy - INFO - Chain [1] done processing
00:23:03 - cmdstanpy - INFO - Chain [1] start processing
00:23:04 - cmdstanpy - INFO - Chain [1] done processing
00:23:04 - cmdstanpy - INFO - Chain [1] start processing
00:23:04 - cmdstanpy - INFO - Chain [1] done processing
00:23:04 - cmdstanpy - INFO - Chain [1] start processing
00:23:04 - cmdstanpy - INFO - Chain [1]

00:23:16 - cmdstanpy - INFO - Chain [1] done processing
00:23:16 - cmdstanpy - INFO - Chain [1] start processing
00:23:16 - cmdstanpy - INFO - Chain [1] done processing
00:23:17 - cmdstanpy - INFO - Chain [1] start processing
00:23:17 - cmdstanpy - INFO - Chain [1] done processing
00:23:17 - cmdstanpy - INFO - Chain [1] start processing
00:23:17 - cmdstanpy - INFO - Chain [1] done processing
00:23:17 - cmdstanpy - INFO - Chain [1] start processing
00:23:17 - cmdstanpy - INFO - Chain [1] done processing
00:23:17 - cmdstanpy - INFO - Chain [1] start processing
00:23:17 - cmdstanpy - INFO - Chain [1] done processing
00:23:17 - cmdstanpy - INFO - Chain [1] start processing
00:23:18 - cmdstanpy - INFO - Chain [1] done processing
00:23:18 - cmdstanpy - INFO - Chain [1] start processing
00:23:18 - cmdstanpy - INFO - Chain [1] done processing
00:23:18 - cmdstanpy - INFO - Chain [1] start processing
00:23:18 - cmdstanpy - INFO - Chain [1] done processing
00:23:18 - cmdstanpy - INFO - Chain [1] 

00:23:30 - cmdstanpy - INFO - Chain [1] done processing
00:23:30 - cmdstanpy - INFO - Chain [1] start processing
00:23:30 - cmdstanpy - INFO - Chain [1] done processing
00:23:30 - cmdstanpy - INFO - Chain [1] start processing
00:23:30 - cmdstanpy - INFO - Chain [1] done processing
00:23:30 - cmdstanpy - INFO - Chain [1] start processing
00:23:30 - cmdstanpy - INFO - Chain [1] done processing
00:23:30 - cmdstanpy - INFO - Chain [1] start processing
00:23:31 - cmdstanpy - INFO - Chain [1] done processing
00:23:31 - cmdstanpy - INFO - Chain [1] start processing
00:23:31 - cmdstanpy - INFO - Chain [1] done processing
00:23:31 - cmdstanpy - INFO - Chain [1] start processing
00:23:31 - cmdstanpy - INFO - Chain [1] done processing
00:23:31 - cmdstanpy - INFO - Chain [1] start processing
00:23:31 - cmdstanpy - INFO - Chain [1] done processing
00:23:31 - cmdstanpy - INFO - Chain [1] start processing
00:23:31 - cmdstanpy - INFO - Chain [1] done processing
00:23:31 - cmdstanpy - INFO - Chain [1] 

00:23:43 - cmdstanpy - INFO - Chain [1] done processing
00:23:43 - cmdstanpy - INFO - Chain [1] start processing
00:23:43 - cmdstanpy - INFO - Chain [1] done processing
00:23:43 - cmdstanpy - INFO - Chain [1] start processing
00:23:43 - cmdstanpy - INFO - Chain [1] done processing
00:23:43 - cmdstanpy - INFO - Chain [1] start processing
00:23:43 - cmdstanpy - INFO - Chain [1] done processing
00:23:43 - cmdstanpy - INFO - Chain [1] start processing
00:23:44 - cmdstanpy - INFO - Chain [1] done processing
00:23:44 - cmdstanpy - INFO - Chain [1] start processing
00:23:44 - cmdstanpy - INFO - Chain [1] done processing
00:23:44 - cmdstanpy - INFO - Chain [1] start processing
00:23:44 - cmdstanpy - INFO - Chain [1] done processing
00:23:44 - cmdstanpy - INFO - Chain [1] start processing
00:23:44 - cmdstanpy - INFO - Chain [1] done processing
00:23:44 - cmdstanpy - INFO - Chain [1] start processing
00:23:44 - cmdstanpy - INFO - Chain [1] done processing
00:23:44 - cmdstanpy - INFO - Chain [1] 

00:23:56 - cmdstanpy - INFO - Chain [1] done processing
00:23:56 - cmdstanpy - INFO - Chain [1] start processing
00:23:56 - cmdstanpy - INFO - Chain [1] done processing
00:23:56 - cmdstanpy - INFO - Chain [1] start processing
00:23:56 - cmdstanpy - INFO - Chain [1] done processing
00:23:56 - cmdstanpy - INFO - Chain [1] start processing
00:23:56 - cmdstanpy - INFO - Chain [1] done processing
00:23:57 - cmdstanpy - INFO - Chain [1] start processing
00:23:57 - cmdstanpy - INFO - Chain [1] done processing
00:23:57 - cmdstanpy - INFO - Chain [1] start processing
00:23:57 - cmdstanpy - INFO - Chain [1] done processing
00:23:57 - cmdstanpy - INFO - Chain [1] start processing
00:23:57 - cmdstanpy - INFO - Chain [1] done processing
00:23:57 - cmdstanpy - INFO - Chain [1] start processing
00:23:57 - cmdstanpy - INFO - Chain [1] done processing
00:23:57 - cmdstanpy - INFO - Chain [1] start processing
00:23:57 - cmdstanpy - INFO - Chain [1] done processing
00:23:57 - cmdstanpy - INFO - Chain [1] 

00:24:09 - cmdstanpy - INFO - Chain [1] done processing
00:24:09 - cmdstanpy - INFO - Chain [1] start processing
00:24:09 - cmdstanpy - INFO - Chain [1] done processing
00:24:10 - cmdstanpy - INFO - Chain [1] start processing
00:24:10 - cmdstanpy - INFO - Chain [1] done processing
00:24:10 - cmdstanpy - INFO - Chain [1] start processing
00:24:10 - cmdstanpy - INFO - Chain [1] done processing
00:24:10 - cmdstanpy - INFO - Chain [1] start processing
00:24:10 - cmdstanpy - INFO - Chain [1] done processing
00:24:10 - cmdstanpy - INFO - Chain [1] start processing
00:24:10 - cmdstanpy - INFO - Chain [1] done processing
00:24:10 - cmdstanpy - INFO - Chain [1] start processing
00:24:10 - cmdstanpy - INFO - Chain [1] done processing
00:24:10 - cmdstanpy - INFO - Chain [1] start processing
00:24:11 - cmdstanpy - INFO - Chain [1] done processing
00:24:11 - cmdstanpy - INFO - Chain [1] start processing
00:24:11 - cmdstanpy - INFO - Chain [1] done processing
00:24:11 - cmdstanpy - INFO - Chain [1] 

00:24:23 - cmdstanpy - INFO - Chain [1] done processing
00:24:23 - cmdstanpy - INFO - Chain [1] start processing
00:24:23 - cmdstanpy - INFO - Chain [1] done processing
00:24:23 - cmdstanpy - INFO - Chain [1] start processing
00:24:23 - cmdstanpy - INFO - Chain [1] done processing
00:24:23 - cmdstanpy - INFO - Chain [1] start processing
00:24:23 - cmdstanpy - INFO - Chain [1] done processing
00:24:23 - cmdstanpy - INFO - Chain [1] start processing
00:24:23 - cmdstanpy - INFO - Chain [1] done processing
00:24:23 - cmdstanpy - INFO - Chain [1] start processing
00:24:24 - cmdstanpy - INFO - Chain [1] done processing
00:24:24 - cmdstanpy - INFO - Chain [1] start processing
00:24:24 - cmdstanpy - INFO - Chain [1] done processing
00:24:24 - cmdstanpy - INFO - Chain [1] start processing
00:24:24 - cmdstanpy - INFO - Chain [1] done processing
00:24:24 - cmdstanpy - INFO - Chain [1] start processing
00:24:24 - cmdstanpy - INFO - Chain [1] done processing
00:24:24 - cmdstanpy - INFO - Chain [1] 

00:24:36 - cmdstanpy - INFO - Chain [1] done processing
00:24:36 - cmdstanpy - INFO - Chain [1] start processing
00:24:36 - cmdstanpy - INFO - Chain [1] done processing
00:24:36 - cmdstanpy - INFO - Chain [1] start processing
00:24:36 - cmdstanpy - INFO - Chain [1] done processing
00:24:36 - cmdstanpy - INFO - Chain [1] start processing
00:24:36 - cmdstanpy - INFO - Chain [1] done processing
00:24:37 - cmdstanpy - INFO - Chain [1] start processing
00:24:37 - cmdstanpy - INFO - Chain [1] done processing
00:24:37 - cmdstanpy - INFO - Chain [1] start processing
00:24:37 - cmdstanpy - INFO - Chain [1] done processing
00:24:37 - cmdstanpy - INFO - Chain [1] start processing
00:24:37 - cmdstanpy - INFO - Chain [1] done processing
00:24:37 - cmdstanpy - INFO - Chain [1] start processing
00:24:37 - cmdstanpy - INFO - Chain [1] done processing
00:24:37 - cmdstanpy - INFO - Chain [1] start processing
00:24:37 - cmdstanpy - INFO - Chain [1] done processing
00:24:37 - cmdstanpy - INFO - Chain [1] 

00:24:49 - cmdstanpy - INFO - Chain [1] done processing
00:24:49 - cmdstanpy - INFO - Chain [1] start processing
00:24:49 - cmdstanpy - INFO - Chain [1] done processing
00:24:49 - cmdstanpy - INFO - Chain [1] start processing
00:24:49 - cmdstanpy - INFO - Chain [1] done processing
00:24:49 - cmdstanpy - INFO - Chain [1] start processing
00:24:49 - cmdstanpy - INFO - Chain [1] done processing
00:24:50 - cmdstanpy - INFO - Chain [1] start processing
00:24:50 - cmdstanpy - INFO - Chain [1] done processing
00:24:50 - cmdstanpy - INFO - Chain [1] start processing
00:24:50 - cmdstanpy - INFO - Chain [1] done processing
00:24:50 - cmdstanpy - INFO - Chain [1] start processing
00:24:50 - cmdstanpy - INFO - Chain [1] done processing
00:24:50 - cmdstanpy - INFO - Chain [1] start processing
00:24:50 - cmdstanpy - INFO - Chain [1] done processing
00:24:50 - cmdstanpy - INFO - Chain [1] start processing
00:24:50 - cmdstanpy - INFO - Chain [1] done processing
00:24:50 - cmdstanpy - INFO - Chain [1] 

In [27]:
forecasts

,ds,yhat,SKU
0,2023-03-04,83.387358,4898Q
0,2022-12-10,145.097231,4870Q
0,2023-05-20,0.089883,4367Q
0,2023-05-20,2759.865665,4215R
0,2022-12-17,14.773010,4415R
...,...,...,...
0,2023-05-20,22.974431,4764R
0,2023-05-20,291.245795,4223R
0,2023-05-20,1054.081821,4932P
0,2023-05-20,1218.748168,4727R


In [28]:
forecasts[forecasts['SKU']=='4344R']

,ds,yhat,SKU
0,2023-04-15,259.258669,4344R


## FORECASTS FOR INVENTORY ACROSS ALL LOCATIONS FOR ALL SKUs

In [29]:
forecasts_1 = pd.DataFrame()

In [30]:
for sku in n1['SKU'].unique():
    # Filter the data for the specific SKU
    sku_data = n1[n1['SKU'] == sku]

    # Prepare the data for Prophet
    df_2 = sku_data.rename(columns={"AS_OF_DATE": "ds", "TTL_INV_LOC": "y"})
    df_2['ds'] = pd.to_datetime(df_2['ds'])
    df_2.sort_values('ds', inplace=True)

    # Create a Prophet model and fit it to the data
    model = Prophet()
    model.fit(df_2)

    # Generate future dates for forecasting (next 4 weeks)
    last_date = df_2['ds'].max()
    next_date = last_date + pd.DateOffset(weeks=1)
    nd1 = pd.DataFrame({'ds':[next_date]})

    # Perform the forecasting
    forecast = model.predict(nd1)

    # Select only the forecasted values for the next 4 weeks
    forecast = forecast[['ds', 'yhat']]

    # Add the SKU-specific forecast to the overall forecasts DataFrame
    forecast['SKU'] = sku
    forecasts_1 = pd.concat([forecasts_1, forecast])

00:24:54 - cmdstanpy - INFO - Chain [1] start processing
00:24:54 - cmdstanpy - INFO - Chain [1] done processing
00:24:54 - cmdstanpy - INFO - Chain [1] start processing
00:24:54 - cmdstanpy - INFO - Chain [1] done processing
00:24:54 - cmdstanpy - INFO - Chain [1] start processing
00:24:54 - cmdstanpy - INFO - Chain [1] done processing
00:24:54 - cmdstanpy - INFO - Chain [1] start processing
00:24:55 - cmdstanpy - INFO - Chain [1] done processing
00:24:55 - cmdstanpy - INFO - Chain [1] start processing
00:24:55 - cmdstanpy - INFO - Chain [1] done processing
00:24:55 - cmdstanpy - INFO - Chain [1] start processing
00:24:55 - cmdstanpy - INFO - Chain [1] done processing
00:24:55 - cmdstanpy - INFO - Chain [1] start processing
00:24:55 - cmdstanpy - INFO - Chain [1] done processing
00:24:55 - cmdstanpy - INFO - Chain [1] start processing
00:24:55 - cmdstanpy - INFO - Chain [1] done processing
00:24:55 - cmdstanpy - INFO - Chain [1] start processing
00:24:55 - cmdstanpy - INFO - Chain [1]

00:25:09 - cmdstanpy - INFO - Chain [1] done processing
00:25:09 - cmdstanpy - INFO - Chain [1] start processing
00:25:09 - cmdstanpy - INFO - Chain [1] done processing
00:25:09 - cmdstanpy - INFO - Chain [1] start processing
00:25:10 - cmdstanpy - INFO - Chain [1] done processing
00:25:10 - cmdstanpy - INFO - Chain [1] start processing
00:25:10 - cmdstanpy - INFO - Chain [1] done processing
00:25:10 - cmdstanpy - INFO - Chain [1] start processing
00:25:10 - cmdstanpy - INFO - Chain [1] done processing
00:25:10 - cmdstanpy - INFO - Chain [1] start processing
00:25:10 - cmdstanpy - INFO - Chain [1] done processing
00:25:10 - cmdstanpy - INFO - Chain [1] start processing
00:25:11 - cmdstanpy - INFO - Chain [1] done processing
00:25:11 - cmdstanpy - INFO - Chain [1] start processing
00:25:11 - cmdstanpy - INFO - Chain [1] done processing
00:25:11 - cmdstanpy - INFO - Chain [1] start processing
00:25:11 - cmdstanpy - INFO - Chain [1] done processing
00:25:11 - cmdstanpy - INFO - Chain [1] 

00:25:24 - cmdstanpy - INFO - Chain [1] done processing
00:25:24 - cmdstanpy - INFO - Chain [1] start processing
00:25:25 - cmdstanpy - INFO - Chain [1] done processing
00:25:25 - cmdstanpy - INFO - Chain [1] start processing
00:25:25 - cmdstanpy - INFO - Chain [1] done processing
00:25:25 - cmdstanpy - INFO - Chain [1] start processing
00:25:25 - cmdstanpy - INFO - Chain [1] done processing
00:25:25 - cmdstanpy - INFO - Chain [1] start processing
00:25:25 - cmdstanpy - INFO - Chain [1] done processing
00:25:25 - cmdstanpy - INFO - Chain [1] start processing
00:25:26 - cmdstanpy - INFO - Chain [1] done processing
00:25:26 - cmdstanpy - INFO - Chain [1] start processing
00:25:26 - cmdstanpy - INFO - Chain [1] done processing
00:25:26 - cmdstanpy - INFO - Chain [1] start processing
00:25:26 - cmdstanpy - INFO - Chain [1] done processing
00:25:26 - cmdstanpy - INFO - Chain [1] start processing
00:25:26 - cmdstanpy - INFO - Chain [1] done processing
00:25:26 - cmdstanpy - INFO - Chain [1] 

00:25:40 - cmdstanpy - INFO - Chain [1] done processing
00:25:40 - cmdstanpy - INFO - Chain [1] start processing
00:25:40 - cmdstanpy - INFO - Chain [1] done processing
00:25:40 - cmdstanpy - INFO - Chain [1] start processing
00:25:40 - cmdstanpy - INFO - Chain [1] done processing
00:25:40 - cmdstanpy - INFO - Chain [1] start processing
00:25:40 - cmdstanpy - INFO - Chain [1] done processing
00:25:40 - cmdstanpy - INFO - Chain [1] start processing
00:25:40 - cmdstanpy - INFO - Chain [1] done processing
00:25:41 - cmdstanpy - INFO - Chain [1] start processing
00:25:41 - cmdstanpy - INFO - Chain [1] done processing
00:25:41 - cmdstanpy - INFO - Chain [1] start processing
00:25:41 - cmdstanpy - INFO - Chain [1] done processing
00:25:41 - cmdstanpy - INFO - Chain [1] start processing
00:25:41 - cmdstanpy - INFO - Chain [1] done processing
00:25:41 - cmdstanpy - INFO - Chain [1] start processing
00:25:41 - cmdstanpy - INFO - Chain [1] done processing
00:25:41 - cmdstanpy - INFO - Chain [1] 

00:25:55 - cmdstanpy - INFO - Chain [1] done processing
00:25:55 - cmdstanpy - INFO - Chain [1] start processing
00:25:55 - cmdstanpy - INFO - Chain [1] done processing
00:25:55 - cmdstanpy - INFO - Chain [1] start processing
00:25:55 - cmdstanpy - INFO - Chain [1] done processing
00:25:55 - cmdstanpy - INFO - Chain [1] start processing
00:25:55 - cmdstanpy - INFO - Chain [1] done processing
00:25:56 - cmdstanpy - INFO - Chain [1] start processing
00:25:56 - cmdstanpy - INFO - Chain [1] done processing
00:25:56 - cmdstanpy - INFO - Chain [1] start processing
00:25:56 - cmdstanpy - INFO - Chain [1] done processing
00:25:56 - cmdstanpy - INFO - Chain [1] start processing
00:25:56 - cmdstanpy - INFO - Chain [1] done processing
00:25:56 - cmdstanpy - INFO - Chain [1] start processing
00:25:56 - cmdstanpy - INFO - Chain [1] done processing
00:25:56 - cmdstanpy - INFO - Chain [1] start processing
00:25:56 - cmdstanpy - INFO - Chain [1] done processing
00:25:57 - cmdstanpy - INFO - Chain [1] 

00:26:10 - cmdstanpy - INFO - Chain [1] done processing
00:26:10 - cmdstanpy - INFO - Chain [1] start processing
00:26:10 - cmdstanpy - INFO - Chain [1] done processing
00:26:11 - cmdstanpy - INFO - Chain [1] start processing
00:26:11 - cmdstanpy - INFO - Chain [1] done processing
00:26:11 - cmdstanpy - INFO - Chain [1] start processing
00:26:11 - cmdstanpy - INFO - Chain [1] done processing
00:26:11 - cmdstanpy - INFO - Chain [1] start processing
00:26:11 - cmdstanpy - INFO - Chain [1] done processing
00:26:11 - cmdstanpy - INFO - Chain [1] start processing
00:26:11 - cmdstanpy - INFO - Chain [1] done processing
00:26:11 - cmdstanpy - INFO - Chain [1] start processing
00:26:11 - cmdstanpy - INFO - Chain [1] done processing
00:26:11 - cmdstanpy - INFO - Chain [1] start processing
00:26:12 - cmdstanpy - INFO - Chain [1] done processing
00:26:12 - cmdstanpy - INFO - Chain [1] start processing
00:26:12 - cmdstanpy - INFO - Chain [1] done processing
00:26:12 - cmdstanpy - INFO - Chain [1] 

00:26:25 - cmdstanpy - INFO - Chain [1] done processing
00:26:25 - cmdstanpy - INFO - Chain [1] start processing
00:26:25 - cmdstanpy - INFO - Chain [1] done processing
00:26:25 - cmdstanpy - INFO - Chain [1] start processing
00:26:25 - cmdstanpy - INFO - Chain [1] done processing
00:26:25 - cmdstanpy - INFO - Chain [1] start processing
00:26:25 - cmdstanpy - INFO - Chain [1] done processing
00:26:26 - cmdstanpy - INFO - Chain [1] start processing
00:26:26 - cmdstanpy - INFO - Chain [1] done processing
00:26:26 - cmdstanpy - INFO - Chain [1] start processing
00:26:26 - cmdstanpy - INFO - Chain [1] done processing
00:26:26 - cmdstanpy - INFO - Chain [1] start processing
00:26:26 - cmdstanpy - INFO - Chain [1] done processing
00:26:26 - cmdstanpy - INFO - Chain [1] start processing
00:26:26 - cmdstanpy - INFO - Chain [1] done processing
00:26:27 - cmdstanpy - INFO - Chain [1] start processing
00:26:27 - cmdstanpy - INFO - Chain [1] done processing
00:26:27 - cmdstanpy - INFO - Chain [1] 

00:26:40 - cmdstanpy - INFO - Chain [1] done processing
00:26:40 - cmdstanpy - INFO - Chain [1] start processing
00:26:40 - cmdstanpy - INFO - Chain [1] done processing
00:26:41 - cmdstanpy - INFO - Chain [1] start processing
00:26:41 - cmdstanpy - INFO - Chain [1] done processing
00:26:41 - cmdstanpy - INFO - Chain [1] start processing
00:26:41 - cmdstanpy - INFO - Chain [1] done processing
00:26:41 - cmdstanpy - INFO - Chain [1] start processing
00:26:41 - cmdstanpy - INFO - Chain [1] done processing
00:26:41 - cmdstanpy - INFO - Chain [1] start processing
00:26:41 - cmdstanpy - INFO - Chain [1] done processing
00:26:41 - cmdstanpy - INFO - Chain [1] start processing
00:26:41 - cmdstanpy - INFO - Chain [1] done processing
00:26:42 - cmdstanpy - INFO - Chain [1] start processing
00:26:42 - cmdstanpy - INFO - Chain [1] done processing
00:26:42 - cmdstanpy - INFO - Chain [1] start processing
00:26:42 - cmdstanpy - INFO - Chain [1] done processing
00:26:42 - cmdstanpy - INFO - Chain [1] 

00:26:56 - cmdstanpy - INFO - Chain [1] done processing
00:26:56 - cmdstanpy - INFO - Chain [1] start processing
00:26:56 - cmdstanpy - INFO - Chain [1] done processing
00:26:56 - cmdstanpy - INFO - Chain [1] start processing
00:26:56 - cmdstanpy - INFO - Chain [1] done processing
00:26:56 - cmdstanpy - INFO - Chain [1] start processing
00:26:56 - cmdstanpy - INFO - Chain [1] done processing
00:26:56 - cmdstanpy - INFO - Chain [1] start processing
00:26:57 - cmdstanpy - INFO - Chain [1] done processing
00:26:57 - cmdstanpy - INFO - Chain [1] start processing
00:26:57 - cmdstanpy - INFO - Chain [1] done processing
00:26:57 - cmdstanpy - INFO - Chain [1] start processing
00:26:57 - cmdstanpy - INFO - Chain [1] done processing
00:26:57 - cmdstanpy - INFO - Chain [1] start processing
00:26:57 - cmdstanpy - INFO - Chain [1] done processing
00:26:57 - cmdstanpy - INFO - Chain [1] start processing
00:26:57 - cmdstanpy - INFO - Chain [1] done processing
00:26:57 - cmdstanpy - INFO - Chain [1] 

In [31]:
forecasts_1

,ds,yhat,SKU
0,2023-03-04,8880.697468,4898Q
0,2022-12-10,8321.590659,4870Q
0,2023-05-20,1.515535,4367Q
0,2023-05-20,11644.340177,4215R
0,2022-12-17,2709.177269,4415R
...,...,...,...
0,2023-05-20,3198.577990,4764R
0,2023-05-20,4022.845001,4223R
0,2023-05-20,13449.887678,4932P
0,2023-05-20,10149.074707,4727R


In [32]:
forecasts_1[forecasts_1['SKU']=='4344R']

,ds,yhat,SKU
0,2023-04-15,8874.605407,4344R


In [33]:
df1 = n1.loc[:,['AS_OF_DATE','SKU','WEEKLY_SALES','TTL_INV_LOC']]
df1

,AS_OF_DATE,SKU,WEEKLY_SALES,TTL_INV_LOC
17,2022-10-08,4898Q,157,8822
18,2022-10-15,4898Q,187,8873
19,2022-10-22,4898Q,168,8849
20,2022-10-29,4898Q,141,8864
21,2022-11-05,4898Q,117,9075
...,...,...,...,...
23040,2023-03-11,4344R,244,8818
23041,2023-03-18,4344R,292,8779
23042,2023-03-25,4344R,261,8765
23043,2023-04-01,4344R,289,8941


In [34]:
df1['SKU'].nunique()

610

In [35]:
n1 = n1.rename(columns={'AS_OF_DATE':'ds'})

In [36]:
df2 = df1.rename(columns={'AS_OF_DATE':'ds'})

In [37]:
df2

,ds,SKU,WEEKLY_SALES,TTL_INV_LOC
17,2022-10-08,4898Q,157,8822
18,2022-10-15,4898Q,187,8873
19,2022-10-22,4898Q,168,8849
20,2022-10-29,4898Q,141,8864
21,2022-11-05,4898Q,117,9075
...,...,...,...,...
23040,2023-03-11,4344R,244,8818
23041,2023-03-18,4344R,292,8779
23042,2023-03-25,4344R,261,8765
23043,2023-04-01,4344R,289,8941


In [38]:
f1 = forecasts.rename(columns={'yhat':'WEEKLY_SALES_FORECAST'})

In [39]:
f1

,ds,WEEKLY_SALES_FORECAST,SKU
0,2023-03-04,83.387358,4898Q
0,2022-12-10,145.097231,4870Q
0,2023-05-20,0.089883,4367Q
0,2023-05-20,2759.865665,4215R
0,2022-12-17,14.773010,4415R
...,...,...,...
0,2023-05-20,22.974431,4764R
0,2023-05-20,291.245795,4223R
0,2023-05-20,1054.081821,4932P
0,2023-05-20,1218.748168,4727R


In [40]:
f2 = forecasts_1.rename(columns={'yhat':'TTL_INV_LOC_FORECAST'})

In [41]:
f2

,ds,TTL_INV_LOC_FORECAST,SKU
0,2023-03-04,8880.697468,4898Q
0,2022-12-10,8321.590659,4870Q
0,2023-05-20,1.515535,4367Q
0,2023-05-20,11644.340177,4215R
0,2022-12-17,2709.177269,4415R
...,...,...,...
0,2023-05-20,3198.577990,4764R
0,2023-05-20,4022.845001,4223R
0,2023-05-20,13449.887678,4932P
0,2023-05-20,10149.074707,4727R


In [42]:
merged_df = pd.merge(f1, f2, on=['ds', 'SKU'], how='inner')
merged_df

,ds,WEEKLY_SALES_FORECAST,SKU,TTL_INV_LOC_FORECAST
0,2023-03-04,83.387358,4898Q,8880.697468
1,2022-12-10,145.097231,4870Q,8321.590659
2,2023-05-20,0.089883,4367Q,1.515535
3,2023-05-20,2759.865665,4215R,11644.340177
4,2022-12-17,14.773010,4415R,2709.177269
...,...,...,...,...
605,2023-05-20,22.974431,4764R,3198.577990
606,2023-05-20,291.245795,4223R,4022.845001
607,2023-05-20,1054.081821,4932P,13449.887678
608,2023-05-20,1218.748168,4727R,10149.074707


In [43]:
import math
merged_df['TTL_INV_LOC_FORECAST'] = merged_df['TTL_INV_LOC_FORECAST'].apply(lambda x: math.ceil(x))
merged_df['WEEKLY_SALES_FORECAST'] = merged_df['WEEKLY_SALES_FORECAST'].apply(lambda x: math.ceil(x))

In [44]:
df2

,ds,SKU,WEEKLY_SALES,TTL_INV_LOC
17,2022-10-08,4898Q,157,8822
18,2022-10-15,4898Q,187,8873
19,2022-10-22,4898Q,168,8849
20,2022-10-29,4898Q,141,8864
21,2022-11-05,4898Q,117,9075
...,...,...,...,...
23040,2023-03-11,4344R,244,8818
23041,2023-03-18,4344R,292,8779
23042,2023-03-25,4344R,261,8765
23043,2023-04-01,4344R,289,8941


In [45]:
max_dates = df2.groupby('SKU')['ds'].max().reset_index()
max_dates['next_date'] = max_dates['ds'] + pd.DateOffset(weeks=1)
nd3 = pd.DataFrame({'SKU': max_dates['SKU'], 'ds': max_dates['next_date']})
df_concat = pd.concat([df2, nd3])
df_concat

,ds,SKU,WEEKLY_SALES,TTL_INV_LOC
17,2022-10-08,4898Q,157.0,8822.0
18,2022-10-15,4898Q,187.0,8873.0
19,2022-10-22,4898Q,168.0,8849.0
20,2022-10-29,4898Q,141.0,8864.0
21,2022-11-05,4898Q,117.0,9075.0
...,...,...,...,...
605,2023-02-18,4991P,NaN,NaN
606,2023-02-18,4995P,NaN,NaN
607,2023-05-20,4997P,NaN,NaN
608,2023-05-20,60075,NaN,NaN


In [46]:
final = pd.merge(df_concat, merged_df, on=['ds', 'SKU'], how='left')
final

,ds,SKU,WEEKLY_SALES,TTL_INV_LOC,WEEKLY_SALES_FORECAST,TTL_INV_LOC_FORECAST
0,2022-10-08,4898Q,157.0,8822.0,NaN,NaN
1,2022-10-15,4898Q,187.0,8873.0,NaN,NaN
2,2022-10-22,4898Q,168.0,8849.0,NaN,NaN
3,2022-10-29,4898Q,141.0,8864.0,NaN,NaN
4,2022-11-05,4898Q,117.0,9075.0,NaN,NaN
...,...,...,...,...,...,...
14403,2023-02-18,4991P,NaN,NaN,1.0,9.0
14404,2023-02-18,4995P,NaN,NaN,0.0,6.0
14405,2023-05-20,4997P,NaN,NaN,77.0,4372.0
14406,2023-05-20,60075,NaN,NaN,3270.0,42936.0


In [47]:
fool_final = pd.merge(final, n1[['ds', 'SKU', 'SRP','CHANGED_SRP']], on=['ds', 'SKU'], how='left')
fool_final

,ds,SKU,WEEKLY_SALES,TTL_INV_LOC,WEEKLY_SALES_FORECAST,TTL_INV_LOC_FORECAST,SRP,CHANGED_SRP
0,2022-10-08,4898Q,157.0,8822.0,NaN,NaN,29.99,29.99
1,2022-10-15,4898Q,187.0,8873.0,NaN,NaN,29.99,29.99
2,2022-10-22,4898Q,168.0,8849.0,NaN,NaN,29.99,29.99
3,2022-10-29,4898Q,141.0,8864.0,NaN,NaN,29.99,29.99
4,2022-11-05,4898Q,117.0,9075.0,NaN,NaN,29.99,29.99
...,...,...,...,...,...,...,...,...
14403,2023-02-18,4991P,NaN,NaN,1.0,9.0,NaN,NaN
14404,2023-02-18,4995P,NaN,NaN,0.0,6.0,NaN,NaN
14405,2023-05-20,4997P,NaN,NaN,77.0,4372.0,NaN,NaN
14406,2023-05-20,60075,NaN,NaN,3270.0,42936.0,NaN,NaN


In [48]:
c1 = ['ds','SRP','SKU','WEEKLY_SALES','WEEKLY_SALES_FORECAST','TTL_INV_LOC','TTL_INV_LOC_FORECAST']
fool_n_final = fool_final[c1]

In [49]:
fool_n_final

,ds,SRP,SKU,WEEKLY_SALES,WEEKLY_SALES_FORECAST,TTL_INV_LOC,TTL_INV_LOC_FORECAST
0,2022-10-08,29.99,4898Q,157.0,NaN,8822.0,NaN
1,2022-10-15,29.99,4898Q,187.0,NaN,8873.0,NaN
2,2022-10-22,29.99,4898Q,168.0,NaN,8849.0,NaN
3,2022-10-29,29.99,4898Q,141.0,NaN,8864.0,NaN
4,2022-11-05,29.99,4898Q,117.0,NaN,9075.0,NaN
...,...,...,...,...,...,...,...
14403,2023-02-18,NaN,4991P,NaN,1.0,NaN,9.0
14404,2023-02-18,NaN,4995P,NaN,0.0,NaN,6.0
14405,2023-05-20,NaN,4997P,NaN,77.0,NaN,4372.0
14406,2023-05-20,NaN,60075,NaN,3270.0,NaN,42936.0


In [50]:
fool_n_final['WEEKLY_SALES_FORECAST'] = fool_n_final['WEEKLY_SALES_FORECAST'].apply(lambda x: 0 if x < 0 else x)

In [51]:
fool_n_final[fool_n_final['SKU'] == '4683R']

,ds,SRP,SKU,WEEKLY_SALES,WEEKLY_SALES_FORECAST,TTL_INV_LOC,TTL_INV_LOC_FORECAST
13452,2023-01-07,40.0,4683R,0.0,NaN,0.0,NaN
13453,2023-01-21,40.0,4683R,172.0,NaN,1671.0,NaN
13454,2023-01-28,40.0,4683R,295.0,NaN,1995.0,NaN
13455,2023-02-04,40.0,4683R,333.0,NaN,2216.0,NaN
13456,2023-02-11,40.0,4683R,433.0,NaN,2582.0,NaN
13457,2023-02-18,40.0,4683R,468.0,NaN,3110.0,NaN
13458,2023-02-25,40.0,4683R,364.0,NaN,4468.0,NaN
13459,2023-03-04,40.0,4683R,393.0,NaN,5204.0,NaN
13460,2023-03-11,40.0,4683R,381.0,NaN,5748.0,NaN
13461,2023-03-18,40.0,4683R,422.0,NaN,6011.0,NaN


## FINAL DF ALONG WITH FORECASTS

In [52]:
fool_n_final

,ds,SRP,SKU,WEEKLY_SALES,WEEKLY_SALES_FORECAST,TTL_INV_LOC,TTL_INV_LOC_FORECAST
0,2022-10-08,29.99,4898Q,157.0,NaN,8822.0,NaN
1,2022-10-15,29.99,4898Q,187.0,NaN,8873.0,NaN
2,2022-10-22,29.99,4898Q,168.0,NaN,8849.0,NaN
3,2022-10-29,29.99,4898Q,141.0,NaN,8864.0,NaN
4,2022-11-05,29.99,4898Q,117.0,NaN,9075.0,NaN
...,...,...,...,...,...,...,...
14403,2023-02-18,NaN,4991P,NaN,1.0,NaN,9.0
14404,2023-02-18,NaN,4995P,NaN,0.0,NaN,6.0
14405,2023-05-20,NaN,4997P,NaN,77.0,NaN,4372.0
14406,2023-05-20,NaN,60075,NaN,3270.0,NaN,42936.0


In [53]:
v = ['4405R','4407R','4408R','4409R','4410R','4413R','4414R','4415R','4417R','4551R','4554R','4562R','4569R','4570R','4572R',
    '4578R','4580R','4583R','4591R','4683R','4686R','4802Q']

In [54]:
len(v)

22

In [55]:
v1 = fool_n_final[fool_n_final['SKU'].isin(v)]

In [56]:
v1

,ds,SRP,SKU,WEEKLY_SALES,WEEKLY_SALES_FORECAST,TTL_INV_LOC,TTL_INV_LOC_FORECAST
81,2022-10-08,90.0,4415R,27.0,NaN,2305.0,NaN
82,2022-10-15,90.0,4415R,32.0,NaN,2276.0,NaN
83,2022-10-22,90.0,4415R,18.0,NaN,2300.0,NaN
84,2022-10-29,90.0,4415R,15.0,NaN,2297.0,NaN
85,2022-11-05,90.0,4415R,24.0,NaN,2424.0,NaN
...,...,...,...,...,...,...,...
14161,2023-04-08,NaN,4580R,NaN,1.0,NaN,2.0
14174,2023-04-15,NaN,4591R,NaN,1.0,NaN,3.0
14235,2023-05-20,NaN,4683R,NaN,684.0,NaN,12205.0
14236,2023-05-20,NaN,4686R,NaN,1839.0,NaN,9700.0


In [57]:
grouped = v1.groupby('SKU')

for sku, group in grouped:
    first_srp = group['SRP'].iloc[0]
    group['FLAG'] = group.apply(lambda x: 0 if x['SRP'] != first_srp else 1, axis=1)
    v1.loc[group.index, 'FLAG'] = group['FLAG']

v1

,ds,SRP,SKU,WEEKLY_SALES,WEEKLY_SALES_FORECAST,TTL_INV_LOC,TTL_INV_LOC_FORECAST,FLAG
81,2022-10-08,90.0,4415R,27.0,NaN,2305.0,NaN,1.0
82,2022-10-15,90.0,4415R,32.0,NaN,2276.0,NaN,1.0
83,2022-10-22,90.0,4415R,18.0,NaN,2300.0,NaN,1.0
84,2022-10-29,90.0,4415R,15.0,NaN,2297.0,NaN,1.0
85,2022-11-05,90.0,4415R,24.0,NaN,2424.0,NaN,1.0
...,...,...,...,...,...,...,...,...
14161,2023-04-08,NaN,4580R,NaN,1.0,NaN,2.0,0.0
14174,2023-04-15,NaN,4591R,NaN,1.0,NaN,3.0,0.0
14235,2023-05-20,NaN,4683R,NaN,684.0,NaN,12205.0,0.0
14236,2023-05-20,NaN,4686R,NaN,1839.0,NaN,9700.0,0.0


In [58]:
grouped1 = v1.groupby('SKU')
modified_groups = []

# Iterate over each group
for sku, group_df in grouped1:
    # Find the last index of the 'FLAG' column within the group
    last_index = group_df['FLAG'].last_valid_index()
    
    # Set the value at the last index of the 'FLAG' column to NaN
    group_df.at[last_index, 'FLAG'] = pd.NA
    
    # Append the modified group dataframe to the list
    modified_groups.append(group_df)

v2 = pd.concat(modified_groups)
v2    

,ds,SRP,SKU,WEEKLY_SALES,WEEKLY_SALES_FORECAST,TTL_INV_LOC,TTL_INV_LOC_FORECAST,FLAG
90,2022-10-08,40.00,4405R,159.0,NaN,3319.0,NaN,1.0
91,2022-10-15,40.00,4405R,116.0,NaN,3276.0,NaN,1.0
92,2022-10-22,40.00,4405R,112.0,NaN,3201.0,NaN,1.0
93,2022-10-29,40.00,4405R,89.0,NaN,3205.0,NaN,1.0
94,2022-11-05,40.00,4405R,80.0,NaN,3276.0,NaN,1.0
...,...,...,...,...,...,...,...,...
1953,2023-04-01,89.99,4802Q,116.0,NaN,7054.0,NaN,0.0
1954,2023-04-08,89.99,4802Q,101.0,NaN,7027.0,NaN,0.0
1955,2023-05-06,89.99,4802Q,102.0,NaN,7047.0,NaN,0.0
1956,2023-05-13,89.99,4802Q,93.0,NaN,7054.0,NaN,0.0


In [59]:
v2['WEEKLY_SALES'].fillna('-', inplace=True)
v2['WEEKLY_SALES_FORECAST'].fillna('-', inplace=True)
v2['TTL_INV_LOC'].fillna('-', inplace=True)
v2['TTL_INV_LOC_FORECAST'].fillna('-', inplace=True)
v2['FLAG'].fillna('-', inplace=True)

## CALCULATIONS FOR PE

In [60]:
# Create an empty DataFrame to store the results
results = pd.DataFrame(columns=['SKU', 'P1', 'Q1', 'P2', 'Q2', 'PE'])

# Iterate over each unique SKU in v2
for sku in v2['SKU'].unique():
    # Filter the data for the current SKU
    z = v2[v2['SKU'] == sku].copy()

    # Calculate P1, P2, Q2
    zero_index = z[z['FLAG'] == 0].index[0]
    position = zero_index - 1
    z['P1'] = z.loc[position, 'SRP']
    z['P2'] = z.loc[zero_index, 'SRP']
    z['Q2'] = z[z['WEEKLY_SALES'] == '-']['WEEKLY_SALES_FORECAST'].mean()

    # Calculate Q1
    wv = z.at[position, 'WEEKLY_SALES']
    z['Q1'] = wv

    # Calculate PE
    z.loc[z['P1'] < z['P2'], 'PE'] = (z['Q1'] - z['Q2']) / z['Q1'] / ((z['P1'] - z['P2']) / z['P1'])
    z.loc[z['P1'] > z['P2'], 'PE'] = (z['Q2'] - z['Q1']) / z['Q1'] / ((z['P1'] - z['P2']) / z['P1'])

    # Append the results to the main DataFrame
    results = results.append(z[['SKU', 'P1', 'Q1', 'P2', 'Q2', 'PE']], ignore_index=True)


In [61]:
results

,SKU,P1,Q1,P2,Q2,PE
0,4405R,40.00,79.0,20.00,65.0,-0.354430
1,4405R,40.00,79.0,20.00,65.0,-0.354430
2,4405R,40.00,79.0,20.00,65.0,-0.354430
3,4405R,40.00,79.0,20.00,65.0,-0.354430
4,4405R,40.00,79.0,20.00,65.0,-0.354430
...,...,...,...,...,...,...
346,4802Q,67.49,175.0,89.99,170.0,-0.085702
347,4802Q,67.49,175.0,89.99,170.0,-0.085702
348,4802Q,67.49,175.0,89.99,170.0,-0.085702
349,4802Q,67.49,175.0,89.99,170.0,-0.085702


In [62]:
results_final = results.drop_duplicates(subset='SKU', keep='first')
results_final

,SKU,P1,Q1,P2,Q2,PE
0,4405R,40.00,79.0,20.00,65.0,-0.354430
10,4407R,40.00,35.0,20.00,38.0,0.171429
20,4408R,90.00,199.0,45.00,175.0,-0.241206
30,4409R,90.00,30.0,45.00,26.0,-0.266667
40,4410R,80.00,12.0,40.00,10.0,-0.333333
50,4413R,40.00,90.0,20.00,94.0,0.088889
60,4414R,40.00,28.0,20.00,19.0,-0.642857
70,4415R,90.00,9.0,45.00,15.0,1.333333
80,4417R,50.00,67.0,25.00,77.0,0.298507
90,4551R,49.00,16.0,36.75,17.0,0.250000


In [63]:
results[results['SKU'] == '4683R']

,SKU,P1,Q1,P2,Q2,PE
291,4683R,40.0,461.0,45.0,684.0,3.869848
292,4683R,40.0,461.0,45.0,684.0,3.869848
293,4683R,40.0,461.0,45.0,684.0,3.869848
294,4683R,40.0,461.0,45.0,684.0,3.869848
295,4683R,40.0,461.0,45.0,684.0,3.869848
296,4683R,40.0,461.0,45.0,684.0,3.869848
297,4683R,40.0,461.0,45.0,684.0,3.869848
298,4683R,40.0,461.0,45.0,684.0,3.869848
299,4683R,40.0,461.0,45.0,684.0,3.869848
300,4683R,40.0,461.0,45.0,684.0,3.869848


In [64]:
v2[v2['SKU']=='4683R']

,ds,SRP,SKU,WEEKLY_SALES,WEEKLY_SALES_FORECAST,TTL_INV_LOC,TTL_INV_LOC_FORECAST,FLAG
13452,2023-01-07,40.0,4683R,0.0,-,0.0,-,1.0
13453,2023-01-21,40.0,4683R,172.0,-,1671.0,-,1.0
13454,2023-01-28,40.0,4683R,295.0,-,1995.0,-,1.0
13455,2023-02-04,40.0,4683R,333.0,-,2216.0,-,1.0
13456,2023-02-11,40.0,4683R,433.0,-,2582.0,-,1.0
13457,2023-02-18,40.0,4683R,468.0,-,3110.0,-,1.0
13458,2023-02-25,40.0,4683R,364.0,-,4468.0,-,1.0
13459,2023-03-04,40.0,4683R,393.0,-,5204.0,-,1.0
13460,2023-03-11,40.0,4683R,381.0,-,5748.0,-,1.0
13461,2023-03-18,40.0,4683R,422.0,-,6011.0,-,1.0
